In [1]:
# load dataframes for each clustering (1ch vs 6ch)
import pandas as pd
import os
import pickle
from modules import statistics

with open('D:/USC/01_code/INS_clustering_v06/_clustering_v01.pkl', 'rb') as f:
    _clustering = pickle.load(f)

# load 1ch df_demo
df_demo = _clustering.df_demo.copy()
# udate and save df_demo_HI_psm by adding new labels (여기)
df_demo_HI_psm = statistics.get_df_demo_HI_psm()
df_1ch = df_demo_HI_psm.copy()

for idx in df_demo.index:
    con = df_1ch['PSG study Number#'].isin([idx])
    df_1ch.loc[con, 'labels'] = df_demo.loc[idx, 'labels']

# df_1ch = pd.read_csv("D:/USC/01_code/INS_clustering_v06/csv_files/df_demo_HI_psm.csv", encoding='euc-kr')
df_2ch = pd.read_csv("./csv_files/df_demo_HI_psm_updated_labels_2ch.csv", encoding='euc-kr')
# df_6ch = pd.read_csv("./csv_files/df_demo_HI_psm_updated_labels_6ch.csv", encoding='euc-kr')

print(df_1ch.labels.value_counts())
print(df_2ch.labels.value_counts())

df_merged = (
    df_1ch[['PSG study Number#', 'labels']]
    .rename(columns={'labels': 'labels_1ch'})
    .merge(
        df_2ch[['PSG study Number#', 'labels']].rename(columns={'labels': 'labels_2ch'}),
        on='PSG study Number#',
        how='left'
    )
)

df_merged = df_merged.loc[0:681, :] # insomnia patients only
df_merged

Laod df_init.csv ... Successful !!


Load dataframes for PSM...
Load 'df_demo_2000t_16f_healthy_insomnia.csv' and 'scalogram_2000t_16f_healthy_insomnia.npy' (1877, 16, 2000, 1)
Load 'df_demo_2000t_16f_only_insomnia.csv' and 'scalogram_2000t_16f_only_insomnia.npy' (682, 16, 2000, 1)

2    682
1    501
0    181
Name: labels, dtype: int64
2    682
0    477
1    205
Name: labels, dtype: int64


,PSG study Number#,labels_1ch,labels_2ch
0,MRN009,0,1
1,MRN013,1,0
2,PE141028,0,1
3,PE141044,1,0
4,PE141054,1,0
...,...,...,...
677,PE191616,1,0
678,PE191621,0,1
679,PE191624,1,0
680,ST17024,0,1


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics import cohen_kappa_score
from scipy.stats import norm

def compute_cohens_kappa_with_se(labels_A, labels_B):
    """
    Compute Cohen's kappa, standard error (approximation),
    z-statistic and p-value (two-tailed).
    """

    # Confusion matrix
    confusion = pd.crosstab(labels_A, labels_B).values
    n = np.sum(confusion)

    # Observed agreement
    po = np.trace(confusion) / n

    # Expected agreement
    row_marginals = np.sum(confusion, axis=1) / n
    col_marginals = np.sum(confusion, axis=0) / n
    pe = np.sum(row_marginals * col_marginals)

    # Kappa
    kappa = (po - pe) / (1 - pe)

    # Standard error (large sample approximation)
    se = np.sqrt((po * (1 - po)) / (n * (1 - pe) ** 2))

    # z-score
    z = kappa / se

    # two-sided p-value
    p_value = 2 * (1 - norm.cdf(abs(z)))

    return kappa, se, z, p_value


def interpret_ari(ari):
    if ari >= 0.90:
        return "Almost identical clustering"
    elif ari >= 0.80:
        return "Very strong similarity"
    elif ari >= 0.65:
        return "Strong similarity"
    elif ari >= 0.50:
        return "Moderate similarity"
    elif ari >= 0.30:
        return "Weak similarity"
    else:
        return "Low similarity"


def interpret_nmi(nmi):
    if nmi >= 0.90:
        return "Almost identical information structure"
    elif nmi >= 0.75:
        return "High shared information"
    elif nmi >= 0.60:
        return "Moderate information overlap"
    elif nmi >= 0.40:
        return "Weak information overlap"
    else:
        return "Low information overlap"


def interpret_kappa(kappa):
    # Landis & Koch (1977)
    if kappa >= 0.81:
        return "Almost perfect agreement"
    elif kappa >= 0.61:
        return "Substantial agreement"
    elif kappa >= 0.41:
        return "Moderate agreement"
    elif kappa >= 0.21:
        return "Fair agreement"
    elif kappa >= 0.00:
        return "Slight agreement"
    else:
        return "Poor agreement"


def compare_clusterings(df,
                        col_A='label_A',
                        col_B='label_B',
                        flip_B=False):
    
    labels_A = df[col_A].values
    labels_B = df[col_B].values
    
    if flip_B:
        labels_B = 1 - labels_B  # Flip 0 to 1 and 1 to 0 if needed 
    
    # ARI
    ari = adjusted_rand_score(labels_A, labels_B)
    
    # NMI
    nmi = normalized_mutual_info_score(labels_A, labels_B)
    
    # Cohen's kappa with SE and p-value
    kappa, se, z, p_value = compute_cohens_kappa_with_se(labels_A, labels_B)
    
    # Results table
    results = pd.DataFrame({
        "Metric": ["ARI", "NMI", "Cohen_kappa"],
        "Value":  [ari,  nmi,  kappa]
    })
    
    print("\n===== Clustering Similarity Results =====\n")
    print(results)
    
    print("\n----- Cohen's kappa statistical test -----")
    print(f"Kappa: {kappa:.4f}")
    print(f"Standard Error: {se:.6f}")
    print(f"Z-statistic: {z:.4f}")
    print(f"P-value (two-sided): {p_value:.6e}")
    
    print("\n----- Interpretation -----")
    print(f"ARI interpretation: {interpret_ari(ari)}")
    print(f"NMI interpretation: {interpret_nmi(nmi)}")
    print(f"Cohen's kappa interpretation: {interpret_kappa(kappa)}")
    
    if p_value < 0.05:
        print("Kappa agreement is statistically significant (p < 0.05).")
    else:
        print("Kappa agreement is NOT statistically significant (p ≥ 0.05).")
    
    return results


In [3]:
compare_clusterings(df_merged, col_A='labels_1ch', col_B='labels_2ch', flip_B=True)


===== Clustering Similarity Results =====

        Metric     Value
0          ARI  0.728521
1          NMI  0.599642
2  Cohen_kappa  0.826832

----- Cohen's kappa statistical test -----
Kappa: 0.8268
Standard Error: 0.024099
Z-statistic: 34.3098
P-value (two-sided): 0.000000e+00

----- Interpretation -----
ARI interpretation: Strong similarity
NMI interpretation: Weak information overlap
Cohen's kappa interpretation: Almost perfect agreement
Kappa agreement is statistically significant (p < 0.05).


,Metric,Value
0,ARI,0.728521
1,NMI,0.599642
2,Cohen_kappa,0.826832


In [24]:
import pandas as pd
import numpy as np

def create_ari_reference():
    data = [
        [0.90, 1.00, "Almost identical clustering"],
        [0.80, 0.89, "Very strong similarity"],
        [0.65, 0.79, "Strong similarity"],
        [0.50, 0.64, "Moderate similarity"],
        [0.30, 0.49, "Weak similarity"],
        [-1.00, 0.29, "Low similarity"]
    ]
    
    df = pd.DataFrame(data, columns=["Lower Bound", "Upper Bound", "Interpretation"])
    df["Metric"] = "ARI"
    df = df[["Metric", "Lower Bound", "Upper Bound", "Interpretation"]]
    return df


def create_nmi_reference():
    data = [
        [0.90, 1.00, "Almost identical information structure"],
        [0.75, 0.89, "High shared information"],
        [0.60, 0.74, "Moderate information overlap"],
        [0.40, 0.59, "Weak information overlap"],
        [0.00, 0.39, "Low information overlap"]
    ]
    
    df = pd.DataFrame(data, columns=["Lower Bound", "Upper Bound", "Interpretation"])
    df["Metric"] = "NMI"
    df = df[["Metric", "Lower Bound", "Upper Bound", "Interpretation"]]
    return df


def create_kappa_reference():
    data = [
        [0.81, 1.00, "Almost perfect agreement"],
        [0.61, 0.80, "Substantial agreement"],
        [0.41, 0.60, "Moderate agreement"],
        [0.21, 0.40, "Fair agreement"],
        [0.00, 0.20, "Slight agreement"],
        [-1.00, -0.01, "Poor agreement"]
    ]
    
    df = pd.DataFrame(data, columns=["Lower Bound", "Upper Bound", "Interpretation"])
    df["Metric"] = "Cohen's κ"
    df["Source"] = "Landis & Koch (1977)"
    df = df[["Metric", "Lower Bound", "Upper Bound", "Interpretation", "Source"]]
    return df


# 생성
ari_reference = create_ari_reference()
nmi_reference = create_nmi_reference()
kappa_reference = create_kappa_reference()

print("\n===== ARI Reference =====\n")
display(ari_reference)

print("\n===== NMI Reference =====\n")
display(nmi_reference)

print("\n===== Cohen's Kappa Reference =====\n")
display(kappa_reference)



===== ARI Reference =====



,Metric,Lower Bound,Upper Bound,Interpretation
0,ARI,0.90,1.00,Almost identical clustering
1,ARI,0.80,0.89,Very strong similarity
2,ARI,0.65,0.79,Strong similarity
3,ARI,0.50,0.64,Moderate similarity
4,ARI,0.30,0.49,Weak similarity
5,ARI,-1.00,0.29,Low similarity



===== NMI Reference =====



,Metric,Lower Bound,Upper Bound,Interpretation
0,NMI,0.90,1.00,Almost identical information structure
1,NMI,0.75,0.89,High shared information
2,NMI,0.60,0.74,Moderate information overlap
3,NMI,0.40,0.59,Weak information overlap
4,NMI,0.00,0.39,Low information overlap



===== Cohen's Kappa Reference =====



,Metric,Lower Bound,Upper Bound,Interpretation,Source
0,Cohen's κ,0.81,1.00,Almost perfect agreement,Landis & Koch (1977)
1,Cohen's κ,0.61,0.80,Substantial agreement,Landis & Koch (1977)
2,Cohen's κ,0.41,0.60,Moderate agreement,Landis & Koch (1977)
3,Cohen's κ,0.21,0.40,Fair agreement,Landis & Koch (1977)
4,Cohen's κ,0.00,0.20,Slight agreement,Landis & Koch (1977)
5,Cohen's κ,-1.00,-0.01,Poor agreement,Landis & Koch (1977)
